# Newton's Method for Optimisation

Implementing Newton's method for optimisation (using hessian for the update). This converges very quickly for some functions, however this is effectively a stationary point finder, i.e. it is blind to actual minimisation of the objective function, so it could very well converge to a local maximum.

In [1]:
import matplotlib.pyplot as plt
import autograd.numpy as np
from autograd import jacobian
from autograd import elementwise_grad

In [35]:
#takes a numpy array
def f(x):
    return np.array([(x[0]-10.)**2 +(x[1]-4.)**2])
 
def MSE(y,x):
    
    return np.linalg.norm(y-x)# does not converge for this because looks for maximum
    #return (y-x) ** 2 # this does converge as it is equivalent to coordinate-wise newton optimisation

In [30]:
# general class of linesearch iterative methods for continuous optimisation
class LineSearch:
    
    def __init__(self,direction_func,backtracking,step):
        self.direction = direction_func
        self.backtracking = backtracking
        self.step = step
        
        self.sigma = 0.1
        self.tol = 0.00000001
        self.rho = 1/2
        
    #takes function and starting x and returns new point
    def take_step(self,f,x,step):
        new_x = x + step*self.direction(f,x)
        return new_x
    
    def Armijo_test(self,f,x,step,direction):
        a = f(x+step*direction)
        b = f(x) - self.sigma*step*np.dot(direction,direction)
        return (a < b).all()
    
    #finds suitable step length using Armijo rule
    def find_step(self,f,x,step):
        iters =0
        direction = self.direction(f,x)
        while not self.Armijo_test(f,x,step,direction):
            step = self.rho * step
            iters +=1
            if iters > 1000:
                return None
        return step
        
    #minimise f from starting point x
    def find_minimum(self,f,x):
        #executes steps until convergence
        prev_x = 10*10*10
        iters = 0
        step = self.step
        while np.linalg.norm(x-prev_x) > self.tol:
            
            #STEP LENGTH SEARCH
            if self.backtracking :
                step = self.find_step(f,x,step)
            if step == None:
                break
                
            # UPDATE
            prev_x = x
            x = self.take_step(f,x,step)
            
            #ITERATION CHECK
            if iters > 10000:
                print("iteration limit reached")
                return
            else:
                #if iters % 10 == 0:
                print("progress")
                print(x)
                print(f(x))
                print()
                iters +=1
                
        print("converged in %d iterations to" %iters)
        print("x value :", x)
        print("out : ", f(x))

In [31]:
def newton(f,x):
    grad = elementwise_grad(f)
    try:
        inv_hessian = np.linalg.inv(jacobian(grad)(x))
    except:
        inv_hessian = np.linalg.inv(jacobian(grad)(x)+ 0.01*np.identity(x.size))
    
    return np.negative(np.dot(inv_hessian,grad(x)))

In [32]:
newton_optim = LineSearch(newton,backtracking = False, step=1.)

In [33]:
x = np.array([12.,112.])
newton_optim.find_minimum(f,x)

progress
[10.  4.]
[0.]

progress
[10.  4.]
[0.]

converged in 2 iterations to
x value : [10.  4.]
out :  [0.]


In [34]:
start = np.ones(10)
y = np.array([1.,21.,32.,43.,54.,65.,76.,87.,98.,109.])
def MSE_y(x):
    return MSE(y,x)

newton_optim.find_minimum(MSE_y,start)


progress
[1.64971832e+16 1.81469015e+17 2.72203523e+17 3.62938031e+17
 4.53672539e+17 5.44407046e+17 6.35141554e+17 7.25876062e+17
 8.16610570e+17 9.07345077e+17]
1.7781025001808817e+18

progress
[-7.46043123e+31 -8.20647435e+32 -1.23097115e+33 -1.64129487e+33
 -2.05161859e+33 -2.46194231e+33 -2.87226602e+33 -3.28258974e+33
 -3.69291346e+33 -4.10323718e+33]
8.041015996338684e+33

progress
[4.59624406e+47 5.05586847e+48 7.58380270e+48 1.01117369e+49
 1.26396712e+49 1.51676054e+49 1.76955396e+49 2.02234739e+49
 2.27514081e+49 2.52793423e+49]
4.953932403718573e+49

progress
[2.47206331e+63 2.71926964e+64 4.07890446e+64 5.43853928e+64
 6.79817410e+64 8.15780892e+64 9.51744374e+64 1.08770786e+65
 1.22367134e+65 1.35963482e+65]
2.664443917037564e+65

progress
[-1.29767838e+79 -1.42744622e+80 -2.14116933e+80 -2.85489244e+80
 -3.56861555e+80 -4.28233866e+80 -4.99606177e+80 -5.70978488e+80
 -6.42350799e+80 -7.13723110e+80]
1.3986661510743614e+81

progress
[-9.39034433e+94 -1.03293788e+96 -1.549